In [48]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [49]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Piyush
[nltk_data]     Agarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
print(stopwords.words('english')) # stopwords do not carry important meaning

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data pre-processing

In [51]:
#loading the dataset to pandas DataFrame
news_dataset=pd.read_excel("C:\\Users\\Piyush Agarwal\\OneDrive\\Desktop\\web development\\Machine-Learning\\Fake news\\train dataset.xlsx",usecols=["id","title","author","text","label"])

In [52]:
print(news_dataset)

       id                                              title  \
0       0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1       1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2       2                  Why the Truth Might Get You Fired   
3       3  15 Civilians Killed In Single US Airstrike Hav...   
4       4  Iranian woman jailed for fictional unpublished...   
...   ...                                                ...   
1050  806  WAR ON THE STREETS OF PARIS: Armed migrants fi...   
1051  807  Oscar Voter: Meryl Streep Only Nominated for A...   
1052  808                     Texas Enacts ’Anti-Sharia’ Law   
1053  809  Chicago Police Board Chair: Windy City Needs F...   
1054  810  Looking Beyond November 8th - A Song of Oligar...   

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        Daniel J. Flynn  Ever get the feeling your life circles the rou.

In [53]:
news_dataset.dtypes

id        object
title     object
author    object
text      object
label     object
dtype: object

In [54]:
news_dataset.shape

(1055, 5)

In [55]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

id         24
title      63
author    156
text      107
label     143
dtype: int64

In [57]:
#there are missing values but we have enough dataset to train the model

In [88]:
news_dataset=news_dataset.fillna('') # used to fill the cell where there is missing with '' means null
news_dataset = news_dataset.dropna(subset=['label']) #we are not removing fillna from label column because it will cause eroor ahead
news_dataset = news_dataset[news_dataset['label'].isin([0,1,'0','1'])] # removing the missing value label from column label



In [105]:
news_dataset['author'] = news_dataset['author'].astype(str)
news_dataset['title'] = news_dataset['title'].astype(str)
news_dataset['label'] = news_dataset['label'].astype(int)


In [106]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']



In [107]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
1050    wmw_admin WAR ON THE STREETS OF PARIS: Armed m...
1051    Jerome Hudson Oscar Voter: Meryl Streep Only N...
1052          Merrill Hope Texas Enacts ’Anti-Sharia’ Law
1053    AWR Hawkins Chicago Police Board Chair: Windy ...
1054    Hefficide Looking Beyond November 8th - A Song...
Name: content, Length: 804, dtype: object


# Stemming
## Stemming is the process of reducing a word to its root word
### example : actor,actress,acting->act

In [108]:
port_stem=PorterStemmer()

In [109]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content) # remove any other character from string except alphabet
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # passing the words to func to reduce to its root
    #also we have removed the stopwords from it
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content


In [110]:
news_dataset['content']=news_dataset['content'].apply(stemming) # applying the function at content column and storing it

In [111]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
1050    wmw admin war street pari arm migrant fight ru...
1051    jerom hudson oscar voter meryl streep nomin an...
1052               merril hope texa enact anti sharia law
1053    awr hawkin chicago polic board chair windi cit...
1054    hefficid look beyond novemb th song oligarchi ...
Name: content, Length: 804, dtype: object


In [112]:
# seperating the data and label
X=news_dataset['content'].values #.values converts a Pandas Series into a NumPy array.
Y=news_dataset['label'].values

In [113]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire'
 'jessica purkiss civilian kill singl us airstrik identifi'
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri'
 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight'
 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time'
 'excerpt draft script donald trump q ampa black church pastor new york time'
 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time'
 'aaron klein obama organ action partner soro link indivis disrupt trump agenda'
 'chri tomlinson bbc comedi sketch real housew isi caus outrag'
 'amando flavio russian research discov secret nazi militari base

# computer does not understand text so we have to convert all the text

In [114]:
#converting teh textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)


In [115]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8140 stored elements and shape (804, 3314)>
  Coords	Values
  (0, 60)	0.28133203927202194
  (0, 495)	0.33372574954697715
  (0, 603)	0.256970351342001
  (0, 740)	0.31438878998449377
  (0, 780)	0.27398046843956
  (0, 1009)	0.29002710205447285
  (0, 1407)	0.2345554974095617
  (0, 1535)	0.2676122481220335
  (0, 1711)	0.3006689988345053
  (0, 1773)	0.31438878998449377
  (0, 2622)	0.28133203927202194
  (0, 3075)	0.3006689988345053
  (1, 301)	0.3138526240149792
  (1, 381)	0.17039181552598082
  (1, 442)	0.3913897922524154
  (1, 571)	0.22670365711726073
  (1, 736)	0.30137192758815984
  (1, 1123)	0.6598860804363869
  (1, 1374)	0.21724700763843408
  (1, 3263)	0.3072648723026896
  (2, 599)	0.3325883890126419
  (2, 637)	0.4465964446432309
  (2, 1099)	0.3749374808996904
  (2, 1208)	0.36310383933423784
  (2, 1896)	0.466975033631732
  :	:
  (801, 2663)	0.3901306168179337
  (801, 2971)	0.35550113102024694
  (802, 217)	0.25363920139107327
  (

# splitting the dataset to training and test data

In [116]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

# Training the model into logistic regression

In [117]:
model=LogisticRegression()

In [118]:
model.fit(X_train,Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


# Evaluation
## accuracy score

In [119]:
# accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [120]:
print("Accuracy score of training data:",training_data_accuracy)

Accuracy score of training data: 0.9751166407465007


In [121]:
# accuracy score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)


In [122]:
print("Accuracy score of test data:",test_data_accuracy)

Accuracy score of test data: 0.9130434782608695


# Making a Predictive System

In [123]:
X_new=X_test[0] # taken 0 row data from test dataset as model hasnt seen it
prediction=model.predict(X_new)
print(prediction)
if(prediction[0]==0):
    print("The news is real")
else:
    print('The news is fail')
    

[0]
The news is real


In [124]:
print(Y_test[0])

0


In [125]:
# means prediction is correct